# 导入依赖包

In [1]:
import pandas as pd
import csv
import numpy as np
from amp.utils import basic_model_serializer
import amp.data_utils.sequence as du_sequence
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


2025-03-29 17:28:15.371309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 加载模型

In [2]:
bms = basic_model_serializer.BasicModelSerializer()
amp_classifier = bms.load_model('models/amp_classifier')
amp_classifier_model = amp_classifier()
mic_classifier = bms.load_model('models/mic_classifier/')
mic_classifier_model = mic_classifier() 

2025-03-29 17:28:23.305141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-29 17:28:23.336043: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/geniusland/home/xiaoliwei/anaconda3/envs/new_fg_ds/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# 加载数据

In [ ]:
import random; 
valid_data = [''.join(random.choices('ACDEFGHIKLMNPQRSTVWY', k=25)) for i in range(1000)]
pad_seq = du_sequence.pad(du_sequence.to_one_hot(valid_data))

# 预测

In [6]:
pred_amp = amp_classifier_model.predict(pad_seq)
pred_mic = mic_classifier_model.predict(pad_seq)

32/32 [==============================] - 0s 7ms/step


In [10]:
def fun(valid_outputs, num):
    def calculate_sensitivity_specificity(y_true, y_pred):
        conf_matrix = np.zeros((2, 2))
        for i in range(len(y_true)):
            conf_matrix[int(y_true[i])][int(y_pred[i])] += 1

        sensitivity = conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[1][0])
        specificity = conf_matrix[0][0] / (conf_matrix[0][0] + conf_matrix[0][1])
        fpr = conf_matrix[0][1] / (conf_matrix[0][1] + conf_matrix[0][0])

        return sensitivity, specificity, fpr

    valid_outputs = valid_outputs > num
    sensitivity, specificity, fpr = calculate_sensitivity_specificity(np.zeros_like(valid_outputs), valid_outputs)

    print(fpr)

fun(pred_amp, 0.8)
fun(pred_mic, 0.5)

0.364
0.061


/tmp/ipykernel_80925/16416630.py:7: RuntimeWarning: invalid value encountered in double_scalars
  sensitivity = conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[1][0])
